# Análise Exploratória de Dados - Voos do Brasil

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
df = pd.read_csv('BrFlights2.csv', encoding = 'latin1')

In [4]:
df.head()

,Voos,Companhia.Aerea,Codigo.Tipo.Linha,Partida.Prevista,Partida.Real,Chegada.Prevista,Chegada.Real,Situacao.Voo,Codigo.Justificativa,Aeroporto.Origem,...,Estado.Origem,Pais.Origem,Aeroporto.Destino,Cidade.Destino,Estado.Destino,Pais.Destino,LongDest,LatDest,LongOrig,LatOrig
0,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-30T08:58:00Z,2016-01-30T08:58:00Z,2016-01-30T10:35:00Z,2016-01-30T10:35:00Z,Realizado,NaN,Afonso Pena,...,PR,Brasil,Salgado Filho,Porto Alegre,RS,Brasil,-51.175381,-29.993473,-49.172481,-25.532713
1,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-13T12:13:00Z,2016-01-13T12:13:00Z,2016-01-13T21:30:00Z,2016-01-13T21:30:00Z,Realizado,NaN,Salgado Filho,...,RS,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-51.175381,-29.993473
2,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-29T12:13:00Z,2016-01-29T12:13:00Z,2016-01-29T21:30:00Z,2016-01-29T21:30:00Z,Realizado,NaN,Salgado Filho,...,RS,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-51.175381,-29.993473
3,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-19T12:13:00Z,2016-01-18T12:03:00Z,2016-01-19T21:30:00Z,2016-01-18T20:41:00Z,Realizado,LIBERACAO SERV. TRAFEGO AEREO/ANTECIPACAO,Salgado Filho,...,RS,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-51.175381,-29.993473
4,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-30T12:13:00Z,2016-01-30T12:13:00Z,2016-01-30T21:30:00Z,2016-01-30T21:30:00Z,Realizado,NaN,Salgado Filho,...,RS,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-51.175381,-29.993473


In [5]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,2542509,2542510,2542511,2542512,2542513,2542514,2542515,2542516,2542517,2542518
Voos,AAL - 203,AAL - 203,AAL - 203,AAL - 203,AAL - 203,AAL - 203,AAL - 203,AAL - 203,AAL - 203,AAL - 203,...,UAL - 129,UAL - 129,UAL - 129,UAL - 129,UAL - 129,UAL - 129,UAL - 129,UAL - 129,UAL - 129,UAL - 129
Companhia.Aerea,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,AMERICAN AIRLINES INC,...,UNITED AIRLINES,UNITED AIRLINES,UNITED AIRLINES,UNITED AIRLINES,UNITED AIRLINES,UNITED AIRLINES,UNITED AIRLINES,UNITED AIRLINES,UNITED AIRLINES,UNITED AIRLINES
Codigo.Tipo.Linha,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,...,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional,Internacional
Partida.Prevista,2016-01-30T08:58:00Z,2016-01-13T12:13:00Z,2016-01-29T12:13:00Z,2016-01-19T12:13:00Z,2016-01-30T12:13:00Z,2016-01-03T23:05:00Z,2016-01-05T23:05:00Z,2016-01-18T12:13:00Z,2016-01-22T23:05:00Z,2016-01-15T23:05:00Z,...,2015-12-15T01:25:00Z,2015-12-28T01:25:00Z,2015-12-22T01:25:00Z,2015-12-30T01:25:00Z,2015-12-17T01:25:00Z,2015-12-14T01:25:00Z,2015-12-01T01:25:00Z,2015-12-27T01:25:00Z,2015-12-05T01:25:00Z,2015-12-09T01:25:00Z
Partida.Real,2016-01-30T08:58:00Z,2016-01-13T12:13:00Z,2016-01-29T12:13:00Z,2016-01-18T12:03:00Z,2016-01-30T12:13:00Z,2016-01-03T23:05:00Z,2016-01-05T23:35:00Z,2016-01-18T13:09:00Z,2016-01-22T23:05:00Z,2016-01-15T23:55:00Z,...,2015-12-15T01:25:00Z,2015-12-28T03:00:00Z,2015-12-22T02:07:00Z,2015-12-30T01:25:00Z,2015-12-17T02:02:00Z,2015-12-14T04:31:00Z,2015-12-01T01:25:00Z,2015-12-27T01:25:00Z,2015-12-05T01:21:00Z,2015-12-09T01:25:00Z
Chegada.Prevista,2016-01-30T10:35:00Z,2016-01-13T21:30:00Z,2016-01-29T21:30:00Z,2016-01-19T21:30:00Z,2016-01-30T21:30:00Z,2016-01-04T07:50:00Z,2016-01-06T07:50:00Z,2016-01-18T21:30:00Z,2016-01-23T07:50:00Z,2016-01-16T07:50:00Z,...,2015-12-15T11:19:00Z,2015-12-28T11:19:00Z,2015-12-22T11:19:00Z,2015-12-30T11:19:00Z,2015-12-17T11:19:00Z,2015-12-14T11:19:00Z,2015-12-01T11:19:00Z,2015-12-27T11:19:00Z,2015-12-05T11:19:00Z,2015-12-09T11:19:00Z
Chegada.Real,2016-01-30T10:35:00Z,2016-01-13T21:30:00Z,2016-01-29T21:30:00Z,2016-01-18T20:41:00Z,2016-01-30T21:30:00Z,2016-01-04T07:50:00Z,2016-01-06T08:35:00Z,2016-01-18T22:24:00Z,2016-01-23T07:50:00Z,2016-01-16T08:28:00Z,...,2015-12-15T11:19:00Z,2015-12-28T13:58:00Z,2015-12-22T11:49:00Z,2015-12-30T11:19:00Z,2015-12-17T11:38:00Z,2015-12-14T14:18:00Z,2015-12-01T11:19:00Z,2015-12-27T11:19:00Z,2015-12-05T10:48:00Z,2015-12-09T11:19:00Z
Situacao.Voo,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,...,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado,Realizado
Codigo.Justificativa,NaN,NaN,NaN,LIBERACAO SERV. TRAFEGO AEREO/ANTECIPACAO,NaN,NaN,LIBERACAO SERV. TRAFEGO AEREO/ANTECIPACAO,CONEXAO DE AERONAVE,NaN,ATRASOS NAO ESPECIFICOS - OUTROS,...,NaN,ATRASOS NAO ESPECIFICOS - OUTROS,ATRASOS NAO ESPECIFICOS - OUTROS,NaN,ATRASOS NAO ESPECIFICOS - OUTROS,ATRASOS NAO ESPECIFICOS - OUTROS,NaN,NaN,ATRASOS NAO ESPECIFICOS - OUTROS,NaN
Aeroporto.Origem,Afonso Pena,Salgado Filho,Salgado Filho,Salgado Filho,Salgado Filho,Miami,Miami,Salgado Filho,Miami,Miami,...,Houston,Houston,Houston,Houston,Houston,Houston,Houston,Houston,Houston,Houston


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542519 entries, 0 to 2542518
Data columns (total 21 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Voos                  object 
 1   Companhia.Aerea       object 
 2   Codigo.Tipo.Linha     object 
 3   Partida.Prevista      object 
 4   Partida.Real          object 
 5   Chegada.Prevista      object 
 6   Chegada.Real          object 
 7   Situacao.Voo          object 
 8   Codigo.Justificativa  object 
 9   Aeroporto.Origem      object 
 10  Cidade.Origem         object 
 11  Estado.Origem         object 
 12  Pais.Origem           object 
 13  Aeroporto.Destino     object 
 14  Cidade.Destino        object 
 15  Estado.Destino        object 
 16  Pais.Destino          object 
 17  LongDest              float64
 18  LatDest               float64
 19  LongOrig              float64
 20  LatOrig               float64
dtypes: float64(4), object(17)
memory usage: 407.4+ MB


***

## Ajuste da Estrutura do DataFrame

### Ajuste do nome das coluna

In [7]:
df.columns

Index(['Voos', 'Companhia.Aerea', 'Codigo.Tipo.Linha', 'Partida.Prevista',
       'Partida.Real', 'Chegada.Prevista', 'Chegada.Real', 'Situacao.Voo',
       'Codigo.Justificativa', 'Aeroporto.Origem', 'Cidade.Origem',
       'Estado.Origem', 'Pais.Origem', 'Aeroporto.Destino', 'Cidade.Destino',
       'Estado.Destino', 'Pais.Destino', 'LongDest', 'LatDest', 'LongOrig',
       'LatOrig'],
      dtype='object')

In [8]:
df.columns = [
    'voos', 'companhia', 'tp_voo', 'dt_partida_prev',
    'dt_partida_real', 'dt_chegada_prev', 'dt_chegada_real', 'situacao',
    'justificativa', 'aeroporto_origem', 'cidade_origem','uf_origem',
    'pais_origem', 'aeroporto_destino', 'cidade_destino','uf_destino',
    'pais_destino', 'long_destino', 'lat_destino', 'long_origem','lat_origem'
]

In [9]:
df.columns

Index(['voos', 'companhia', 'tp_voo', 'dt_partida_prev', 'dt_partida_real',
       'dt_chegada_prev', 'dt_chegada_real', 'situacao', 'justificativa',
       'aeroporto_origem', 'cidade_origem', 'uf_origem', 'pais_origem',
       'aeroporto_destino', 'cidade_destino', 'uf_destino', 'pais_destino',
       'long_destino', 'lat_destino', 'long_origem', 'lat_origem'],
      dtype='object')

### Alterando o tipo das colunas de data

In [12]:
# ANTES: '2016-01-30T08:58:00Z'
df['dt_partida_prev'][0]

'2016-01-30T08:58:00Z'

In [14]:
# DEPOIS: 2016-01-30 08:58
pd.to_datetime(df['dt_partida_prev'][0])

Timestamp('2016-01-30 08:58:00+0000', tz='UTC')

In [18]:
pd.to_datetime(df['dt_partida_prev'][0]).strftime('%Y-%m-%d %H:%M')

'2016-01-30 08:58'

In [21]:
df['dt_partida_prev'] = pd.to_datetime(df['dt_partida_prev']).dt.strftime('%Y-%m-%d %H:%M')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542519 entries, 0 to 2542518
Data columns (total 21 columns):
 #   Column             Dtype  
---  ------             -----  
 0   voos               object 
 1   companhia          object 
 2   tp_voo             object 
 3   dt_partida_prev    object 
 4   dt_partida_real    object 
 5   dt_chegada_prev    object 
 6   dt_chegada_real    object 
 7   situacao           object 
 8   justificativa      object 
 9   aeroporto_origem   object 
 10  cidade_origem      object 
 11  uf_origem          object 
 12  pais_origem        object 
 13  aeroporto_destino  object 
 14  cidade_destino     object 
 15  uf_destino         object 
 16  pais_destino       object 
 17  long_destino       float64
 18  lat_destino        float64
 19  long_origem        float64
 20  lat_origem         float64
dtypes: float64(4), object(17)
memory usage: 407.4+ MB


In [23]:
# dt_partida_real
df['dt_partida_real'] = pd.to_datetime(df['dt_partida_real']).dt.strftime('%Y-%m-%d %H:%M')

# dt_chegada_prev 
df['dt_chegada_prev'] = pd.to_datetime(df['dt_chegada_prev']).dt.strftime('%Y-%m-%d %H:%M')

# dt_chegada_real
df['dt_chegada_real'] = pd.to_datetime(df['dt_chegada_real']).dt.strftime('%Y-%m-%d %H:%M')

In [24]:
# Alterar o data type das colunas para DateTime

# dt_partida_prev
df['dt_partida_prev'] = df['dt_partida_prev'].astype('datetime64[ns]')

# dt_partida_real
df['dt_partida_real'] = df['dt_partida_real'].astype('datetime64[ns]')

# dt_chegada_prev 
df['dt_chegada_prev'] = df['dt_chegada_prev'].astype('datetime64[ns]')

# dt_chegada_real
df['dt_chegada_real'] = df['dt_chegada_real'].astype('datetime64[ns]')

### Alterando a coluna situacao

In [25]:
df['situacao'].nunique()

2

In [26]:
df['situacao'].unique()

array(['Realizado', 'Cancelado'], dtype=object)

In [27]:
# VERIFICAR 
# Se a coluna situacao for igual a Realizado
### >>> Atribui 1 como valor
### >>> Se não, atribui 0 como valor

df['situacao']

0          Realizado
1          Realizado
2          Realizado
3          Realizado
4          Realizado
             ...    
2542514    Realizado
2542515    Realizado
2542516    Realizado
2542517    Realizado
2542518    Realizado
Name: situacao, Length: 2542519, dtype: object

In [29]:
df['situacao'] = np.where( df['situacao'] == 'Realizado', 1, 0).astype(int)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542519 entries, 0 to 2542518
Data columns (total 21 columns):
 #   Column             Dtype         
---  ------             -----         
 0   voos               object        
 1   companhia          object        
 2   tp_voo             object        
 3   dt_partida_prev    datetime64[ns]
 4   dt_partida_real    datetime64[ns]
 5   dt_chegada_prev    datetime64[ns]
 6   dt_chegada_real    datetime64[ns]
 7   situacao           int32         
 8   justificativa      object        
 9   aeroporto_origem   object        
 10  cidade_origem      object        
 11  uf_origem          object        
 12  pais_origem        object        
 13  aeroporto_destino  object        
 14  cidade_destino     object        
 15  uf_destino         object        
 16  pais_destino       object        
 17  long_destino       float64       
 18  lat_destino        float64       
 19  long_origem        float64       
 20  lat_origem         float

## Tratando as colunas com registros nulos

In [32]:
df.isna().sum()

voos                       0
companhia                  0
tp_voo                     0
dt_partida_prev            0
dt_partida_real       289196
dt_chegada_prev            0
dt_chegada_real       289196
situacao                   0
justificativa        1510212
aeroporto_origem           0
cidade_origem              0
uf_origem                  0
pais_origem                0
aeroporto_destino          0
cidade_destino             0
uf_destino                 0
pais_destino               0
long_destino               0
lat_destino                0
long_origem                0
lat_origem                 0
dtype: int64

In [37]:
df[ (df['dt_chegada_real'].isnull() == True ) & (df['situacao'] == 0) ]

,voos,companhia,tp_voo,dt_partida_prev,dt_partida_real,dt_chegada_prev,dt_chegada_real,situacao,justificativa,aeroporto_origem,...,uf_origem,pais_origem,aeroporto_destino,cidade_destino,uf_destino,pais_destino,long_destino,lat_destino,long_origem,lat_origem
111,AAL - 213,AMERICAN AIRLINES INC,Internacional,2016-01-18 00:29:00,NaT,2016-01-18 07:56:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Miami,...,N/I,Estados Unidos,Presidente Juscelino Kubitschek,Brasilia,DF,Brasil,-47.917235,-15.869737,-80.287046,25.795865
115,AAL - 213,AMERICAN AIRLINES INC,Internacional,2016-01-17 00:29:00,NaT,2016-01-17 07:56:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Miami,...,N/I,Estados Unidos,Presidente Juscelino Kubitschek,Brasilia,DF,Brasil,-47.917235,-15.869737,-80.287046,25.795865
149,AAL - 214,AMERICAN AIRLINES INC,Internacional,2016-01-15 23:59:00,NaT,2016-01-16 07:46:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Presidente Juscelino Kubitschek,...,DF,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-47.917235,-15.869737
158,AAL - 229,AMERICAN AIRLINES INC,Internacional,2016-01-21 00:34:00,NaT,2016-01-21 08:44:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Miami,...,N/I,Estados Unidos,Deputado Luis Eduardo Magalhaes,Salvador,BA,Brasil,-38.331241,-12.911098,-80.287046,25.795865
162,AAL - 229,AMERICAN AIRLINES INC,Internacional,2016-01-05 00:34:00,NaT,2016-01-05 08:44:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Miami,...,N/I,Estados Unidos,Deputado Luis Eduardo Magalhaes,Salvador,BA,Brasil,-38.331241,-12.911098,-80.287046,25.795865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541976,TAM - 8113,TAM,Internacional,2015-11-28 20:45:00,NaT,2015-11-29 06:25:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Mexico,...,N/I,Mexico,Guarulhos - Governador Andre Franco Montoro,Guarulhos,SP,Brasil,-46.478126,-23.434553,-99.071908,19.436076
2541979,TAM - 8113,TAM,Internacional,2015-11-10 20:45:00,NaT,2015-11-11 06:25:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Mexico,...,N/I,Mexico,Guarulhos - Governador Andre Franco Montoro,Guarulhos,SP,Brasil,-46.478126,-23.434553,-99.071908,19.436076
2541981,TAM - 8113,TAM,Internacional,2015-11-27 20:45:00,NaT,2015-11-28 06:25:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Mexico,...,N/I,Mexico,Guarulhos - Governador Andre Franco Montoro,Guarulhos,SP,Brasil,-46.478126,-23.434553,-99.071908,19.436076
2541993,TAM - 8113,TAM,Internacional,2015-11-19 20:45:00,NaT,2015-11-20 06:25:00,NaT,0,CANCELAMENTO POR MOTIVOS TECNICOS - OPERACIONAIS,Mexico,...,N/I,Mexico,Guarulhos - Governador Andre Franco Montoro,Guarulhos,SP,Brasil,-46.478126,-23.434553,-99.071908,19.436076


In [38]:
# !pip install pandas_profiling

In [39]:
from pandas_profiling import ProfileReport

In [41]:
profile = ProfileReport(
    df.sample(100000),                            # Pega 100k registros aleatórios do DF
    title = 'AED - Voos do Brasil',               # Titulo que irá aparecer no documento HTML
    html = { 'style': {'full_width': True} },     # Queremos que o HTML tenha o stilo para o tamanho real
    explorative = True,                           # Habilita a opção de exploração
    dark_mode = True                              # A cor mais escura

)


In [42]:
profile.to_file('df_voos_profile_dark_mode.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]